### Model specification

#### Priors: 
$$\psi_{1} \sim Unif(0, 1)$$
$$\mu \sim Unif(-10, 10)$$
$$\sigma \sim Unif(0, 5)$$
$$b0, b1, a0, a1 \sim \mathcal{N}(0, 10)$$

#### Linear models
$$ log(\frac{\gamma}{1 - \gamma}) = b0 + b1 * \mathcal{C}_{i}$$
$$ log(\frac{\epsilon}{1 - \epsilon}) = b0 + b1 * \mathcal{L}_{i} $$

#### Likelihood
$$ z_{i, t} \sim Bern((1 - z_{i, t-1})\gamma_{i, t-1} + z_{i, t-1}(1 - \epsilon_{i, t-1})) $$
$$ p_{i, t} \sim \mathcal{N}(\mu, \sigma^{2})$$
$$ y_{i, t} \sim Binom(\#visits, p_{i, t} * z_{i, t})$$

### Discrepancy metric - goodness of fit
$$ \chi^{2} = \sum \frac{(observed - expected)^2}{expected}$$
Tukey-Freeman test statistic
$$ TF = \sum(\sqrt{observed} - \sqrt{expected})^2$$

The ratio between the observed and simulated is computed to check if the model fits the data well.

### Model in BUGS language

In [ ]:
sink("voles.txt")
cat("
model{
  # Prior specification
  psi1 ~ dunif(0, 1)
  mu ~ dunif(-10, 10)
  sigma ~ dunif(0, 5)
  tau <- sigma^(-2)
  
  # priors for the linear model
  b0 ~ dnorm(0, 0.1)
  b1 ~ dnorm(0, 0.1)
  a0 ~ dnorm(0, 0.1)
  a1 ~ dnorm(0, 0.1)
  
  for(site in 1:n.sites) {
    for(year in 2:n.years) {
      logit(gamma[site, year - 1]) <- b0 + b1 * connectivity[site]
      logit(epsilon[site, year - 1]) <- a0 + a1 * length[site]
    }
  }
   
  # Likelihood
  for(site in 1:n.sites){
    z[site, 1] ~ dbern(psi1)
    for(year in 2:n.years) {      
      z[site, year] ~ dbern(
        (1 - z[site, year - 1]) * gamma[site, year - 1] + z[site, year - 1] * (1 - epsilon[site, year - 1])
      )
    }
  }

  for(site in 1:n.sites) {
    for(year in 1:n.years) {
      q[site, year] ~ dnorm(mu, tau)
      logit(p[site, year]) <- q[site, year]
      y[site, year] ~ dbinom(p[site, year] * z[site, year], visits[site, year])
      
      # calculations for goodness of fit and Freeman-Tukey test
      ysim[site, year] ~ dbin(p[site, year] * z[site, year], visits[site, year])
      yexp[site, year] <- p[site, year] * visits[site, year] * z[site, year] + 0.001
      x2.obs[site, year] <- pow((y[site, year] - yexp[site, year]), 2) / (yexp[site, year])    # for observed data
      x2.sim[site, year] <- pow((ysim[site, year] - yexp[site, year]), 2) / (yexp[site, year]) # for 'ideal' data
      ft.obs[site, year] <- pow(sqrt(y[site, year]) - sqrt(yexp[site, year]), 2)               # for observed data
      ft.sim[site, year] <- pow(sqrt(ysim[site, year]) - sqrt(yexp[site, year]), 2)            # for 'ideal' data
    }
  }

  Chi2.obs <- sum(x2.obs[,])
  Chi2.sim <- sum(x2.sim[,])
  Chi2.ratio <- x2.obs/x2.sim
  FT.obs <- sum(ft.obs[,])
  FT.sim <- sum(ft.sim[,])
  FT.ratio <- FT.obs/FT.sim

  for(year in 1:n.years){
    propocc[year] <- sum(z[, year]) / n.sites
  }
}
",fill = TRUE)
sink()

### Model fitting

In [ ]:
visits = as.matrix(voles[,c("j1", "j2", "j3", "j4")])
obs = as.matrix(voles[, c("y1", "y2", "y3", "y4")])

In [ ]:
# Bundle data
volesdata <- list(
    y = obs,
    n.years = 4,
    n.sites = nrow(voles),
    visits = visits,
    connectivity = voles$Connectivity,
    length = voles$Length
)

volesinits <- function() {
  list(psi1 = runif(1, 0, 1),
       mu = runif(1, 0, 1),
       sigma = runif(1, 0, 3),
       b0 = runif(1, 0.4, 0.9),
       b1 = runif(1, 0.4, 0.9),
       a0 = runif(1, 0.1, 0.5),
       a1 = runif(1, 0.7, 0.9),
       z = ifelse(obs>0,1,0)
       )
}

# Parameters monitored
volesparams <- c("gamma", "epsilon", "p", "propocc", 
                 "Chi2.obs", "Chi2.sim", "Chi2.ratio",
                 "FT.obs", "FT.sim", "FT.ratio")

# MCMC settings
ni <- 10000
nt <- 6
nb <- 3000
nc <- 3

volesout <- jags(
    data = volesdata,
    inits = volesinits,
    parameters.to.save = volesparams,
    model.file = "voles.txt",
    n.chains = nc,
    n.iter = ni,
    n.burnin = nb,
    n.thin = nt
)

### Convergence diagnostics

In [ ]:
MCMCtrace(
    volesout,                 #the fitted model
    params = volesparams[3],  #core model parameters
    iter = ni,                #plot all iterations
    pdf = FALSE,              #DON'T write to a PDF
    type = "density",
    ind=TRUE
)

In [ ]:
MCMCsummary(
    volesout,
    params =  volesparams[1:3]) #out parameters of interest

### Occupancy plot

In [ ]:
 occupancy <- data.frame(
    Year=c(2009:2012),
    Estimate=round(volesout$mean$propocc,2),
    Lower=round(volesout$q2.5$propocc,2),
    Upper=round(volesout$q97.5$propocc, 2)
)


ggplot(data = occupancy, aes(x=Year,y=Estimate)) + 
  geom_errorbar(aes(ymin=Lower, ymax=Upper), width=0) +
  geom_point(size=3) +
  theme_bw() + theme(
      text = element_text(size=20),
      panel.background = element_blank()
  ) + scale_x_discrete(limits=occupancy$Year)

occupancy

### Goodness of fit test

In [ ]:
obs.chi <- volesout$sims.list$Chi2.obs
sim.chi <- volesout$sims.list$Chi2.sim
obs.ft <- volesout$sims.list$FT.obs
sim.ft <- volesout$sims.list$FT.sim

options(repr.plot.width=12, repr.plot.height=6)
par(mfrow=c(1,2), oma=c(0,0,0,0), mar=c(4,4,1,1))
plot(sim.chi~obs.chi,col=adjustcolor(ifelse(obs.chi>sim.chi, "darkred","darkorange"),0.05), 
     pch=16, asp=1, xlab="Observed data", ylab="Simulated data",
     xlim=c(50, 150), ylim=c(50, 150), cex=3, cex.lab=1.5, cex.main=1.5)
abline(0,1, lwd=2)
mean(obs.chi>sim.chi)

### Extinction vs Length plot

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
lengthepsilon = data.frame(
    Length = voles$Length,
    Epsilon = volesout$mean$epsilon[,1],
    Lower = volesout$q2.5$epsilon[,1],
    Upper = volesout$q97.5$epsilon[,1]
)

ggplot(data = lengthepsilon, 
       aes(x = Length, y = Epsilon)) +
       geom_line(line_type="dotted", color="red", size=1.5) +
       geom_point(color="darkblue", size=2) +
       geom_ribbon(aes(x=Length, y=Epsilon, ymin=Lower, ymax=Upper),
              fill="grey", alpha = 0.25) +
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), 
        axis.line = element_line(colour = "black"),
        text = element_text(size=20),
        axis.ticks.x=element_blank()) + ylab("Extinction probability")


### Colonization vs connectivity

In [ ]:
options(repr.plot.width=8, repr.plot.height=6) 

options(repr.plot.width=8, repr.plot.height=6)
lengthepsilon = data.frame(
    Connectivity = voles$Connectivity,
    Colonization = volesout$mean$gamma[,1],
    Lower = volesout$q2.5$gamma[,1],
    Upper = volesout$q97.5$gamma[,1]
)

ggplot(data = lengthepsilon, 
       aes(x = Connectivity, y = Colonization)) +
       geom_line(line_type="dotted", color="red", size=1.5) +
       geom_point(color="darkblue", size=2) +
       geom_ribbon(aes(x=Connectivity, y=Colonization, ymin=Lower, ymax=Upper),
              fill="grey", alpha = 0.25) + 
       theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), 
        axis.line = element_line(colour = "black"),
        text = element_text(size=20),
        axis.ticks.x=element_blank()) + ylab("Colonization probability")